In [1]:
from torchvision import models


In [2]:
model = models.resnet18(pretrained=True)

In [3]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [6]:

import PIL
from torchvision import models
from torchvision import transforms, utils
from torchvision import datasets, models, transforms

import os,sys,math
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import torch.utils.model_zoo as model_zoo
from sklearn.manifold import TSNE

import torch.nn as nn

import numpy as np

import torch

import sklearn.manifold

from time import time
import matplotlib.pyplot as plt
from matplotlib import offsetbox


model_urls = {
    'densenet121': None,
    'resnet18': None
}




class dataset_folderdogcat(Dataset):
  def __init__(self, root_dir, trvaltest, perclasslimit, transform=None, offset=0):

    """
    Args:

        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """

    self.root_dir = root_dir

    self.transform = transform
    self.imgfilenames=[]
    self.labels=[]

    limit=perclasslimit

    for root, directories, filenames in os.walk( os.path.join(self.root_dir,'Dog')):
      ct=0
      for filename in filenames:
        ct+=1
        if ct<offset:
          continue
        if ct+2 >=offset+limit:
          break 
        v=os.path.join(root,filename)
        if os.path.isfile(v):
          #print 
          #print(filename)
          self.imgfilenames.append(os.path.join('Dog',filename))
          self.labels.append(0)


    for root, directories, filenames in os.walk( os.path.join(self.root_dir,'Cat')):
      ct=0
      for filename in filenames:
        ct+=1
        if ct+2 >=limit:
          break 
        v=os.path.join(root,filename)
        if os.path.isfile(v):
          #print 
          #print(filename)
          self.imgfilenames.append(os.path.join('Cat',filename))
          self.labels.append(1)

    print(np.mean(np.asarray(self.labels)))

  def __len__(self):
      return len(self.imgfilenames)

  def __getitem__(self, idx):
    image = PIL.Image.open( os.path.join(self.root_dir,self.imgfilenames[idx]))


    #label=self.labels[idx,:].astype(np.float32)


    if self.transform:
      image = self.transform(image)

    # if you do five crop, thne you MUST change this part, as outputs are now 4 d tensors!!!
    if image.size()[0]==1:
      image=image.repeat([3,1,1])
    if image.size()[0]==4:
      image=image[0:3,:,:]

    #print(self.imgfilenames[idx])

    sample = {'image': image,  'filename': self.imgfilenames[idx], 'label': self.labels[idx]}

    return sample

In [27]:
def fcomp():

  pt= 'PetImages/' 
  savedir='dogcatfeats_0'



  if not os.path.isdir(savedir):
    os.makedirs(savedir)

  device=torch.device('cpu')

 
  BATCH_SIZE=32
  imgsize=64

  data_transform = transforms.Compose([
               transforms.Scale(imgsize),
               transforms.CenterCrop(imgsize),
               transforms.ToTensor(),
               transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
              ])

  dataset = dataset_folderdogcat(root_dir=pt,trvaltest=-1, perclasslimit = 200, transform=data_transform , offset=5000)

  print('len(dataset)',len(dataset))

  dataset_loader = torch.utils.data.DataLoader(dataset,batch_size=BATCH_SIZE, shuffle=False, num_workers=1, drop_last=False, pin_memory=True)

  ############################################
  ############################################
  ############################################
  model=models.resnet18(pretrained=True).to(device)
  #your pretrained model.to(device)
  model.eval()
  # print(model)


  for k,dic in enumerate(dataset_loader):
    print(k)

    fnames=dic['filename']
    #print(fnames)

    imgs=dic['image']
    with torch.no_grad():

      ############################################
      ############################################
      ############################################
      # modify your model so that it returns the prediction p and the feature after adaptiveaveragepooling
      
      res18_conv = nn.Sequential(*list(model.children())[:-2])
      fts=res18_conv(imgs.to(device))

      npfts=fts.to('cpu').numpy()

    print(npfts.shape)
    for i,fn in enumerate(fnames):
      savename=os.path.join(savedir, fn+'_ft.npy')
      curdir=  os.path.dirname(savename)
      if not os.path.isdir(curdir):
        os.makedirs(curdir)
      print(savename)
 
      np.save(savename,npfts[i,:])

In [28]:
fcomp()

/Users/lixingxuan/anaconda3/envs/tesnorflow/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


0.49873417721518987
len(dataset) 395
0
(32, 512, 2, 2)
dogcatfeats_0/Dog/3870.jpg_ft.npy
dogcatfeats_0/Dog/6908.jpg_ft.npy
dogcatfeats_0/Dog/8925.jpg_ft.npy
dogcatfeats_0/Dog/9385.jpg_ft.npy
dogcatfeats_0/Dog/12180.jpg_ft.npy
dogcatfeats_0/Dog/1901.jpg_ft.npy
dogcatfeats_0/Dog/11489.jpg_ft.npy
dogcatfeats_0/Dog/10797.jpg_ft.npy
dogcatfeats_0/Dog/4879.jpg_ft.npy
dogcatfeats_0/Dog/10218.jpg_ft.npy
dogcatfeats_0/Dog/5030.jpg_ft.npy
dogcatfeats_0/Dog/2987.jpg_ft.npy
dogcatfeats_0/Dog/3441.jpg_ft.npy
dogcatfeats_0/Dog/11106.jpg_ft.npy
dogcatfeats_0/Dog/7627.jpg_ft.npy
dogcatfeats_0/Dog/8514.jpg_ft.npy
dogcatfeats_0/Dog/6539.jpg_ft.npy
dogcatfeats_0/Dog/1256.jpg_ft.npy
dogcatfeats_0/Dog/1530.jpg_ft.npy
dogcatfeats_0/Dog/7141.jpg_ft.npy
dogcatfeats_0/Dog/8272.jpg_ft.npy
dogcatfeats_0/Dog/12369.jpg_ft.npy
dogcatfeats_0/Dog/3327.jpg_ft.npy
dogcatfeats_0/Dog/11660.jpg_ft.npy
dogcatfeats_0/Dog/4448.jpg_ft.npy
dogcatfeats_0/Dog/5756.jpg_ft.npy
dogcatfeats_0/Dog/2039.jpg_ft.npy
dogcatfeats_0/Dog/33

(32, 512, 2, 2)
dogcatfeats_0/Cat/6428.jpg_ft.npy
dogcatfeats_0/Cat/1347.jpg_ft.npy
dogcatfeats_0/Cat/7736.jpg_ft.npy
dogcatfeats_0/Cat/8405.jpg_ft.npy
dogcatfeats_0/Cat/2896.jpg_ft.npy
dogcatfeats_0/Cat/3550.jpg_ft.npy
dogcatfeats_0/Cat/11017.jpg_ft.npy
dogcatfeats_0/Cat/10309.jpg_ft.npy
dogcatfeats_0/Cat/5121.jpg_ft.npy
dogcatfeats_0/Cat/9928.jpg_ft.npy
dogcatfeats_0/Cat/638.jpg_ft.npy
dogcatfeats_0/Cat/7905.jpg_ft.npy
dogcatfeats_0/Cat/9096.jpg_ft.npy
dogcatfeats_0/Cat/8388.jpg_ft.npy
dogcatfeats_0/Cat/12293.jpg_ft.npy
dogcatfeats_0/Cat/5874.jpg_ft.npy
dogcatfeats_0/Cat/11942.jpg_ft.npy
dogcatfeats_0/Cat/10484.jpg_ft.npy
dogcatfeats_0/Cat/5860.jpg_ft.npy
dogcatfeats_0/Cat/11956.jpg_ft.npy
dogcatfeats_0/Cat/10490.jpg_ft.npy
dogcatfeats_0/Cat/9082.jpg_ft.npy
dogcatfeats_0/Cat/12287.jpg_ft.npy
dogcatfeats_0/Cat/7911.jpg_ft.npy
dogcatfeats_0/Cat/2869.jpg_ft.npy
dogcatfeats_0/Cat/9914.jpg_ft.npy
dogcatfeats_0/Cat/7939.jpg_ft.npy
dogcatfeats_0/Cat/1390.jpg_ft.npy
dogcatfeats_0/Cat/604.jpg

In [24]:
print(res18_conv)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, ker